# 📊 Evaluation Script

Let's start by reading the data

In [42]:
import os
import cv2
import numpy as np
import pickle
from sklearn.metrics.pairwise import pairwise_distances_argmin_min
from sklearn.metrics import accuracy_score
import skimage.io as io

In [43]:
def unsharp_masking(image, blur_radius=5, sharpen_amount=1.0):
    """Apply unsharp masking followed by dilation to enhance image details."""
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(image, (0, 0), blur_radius)
    
    # Apply unsharp masking
    sharpened = cv2.addWeighted(image, 1.0 + sharpen_amount, blurred, -sharpen_amount, 0)
    
    # Ensure the sharpened image doesn't have values below the original image
    sharpened = np.where(image >= blurred, sharpened, image)
    
    return sharpened

def remove_noise22(image, dilation_kernel=None):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bilateral_filtered = cv2.bilateralFilter(gray, 15, sigmaColor=3, sigmaSpace=10)
    
    median_blurred = cv2.medianBlur(bilateral_filtered, 3)
    result = unsharp_masking(median_blurred, blur_radius=10, sharpen_amount=4) * 255
    
    result = np.uint8(result)
    edges = cv2.Canny(result, 80, 255,apertureSize=3)
       # Optionally apply dilation
    if dilation_kernel is not None:
        # Perform dilation
        edges = cv2.dilate(edges, dilation_kernel)
    return edges

def extract_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return descriptors

In [44]:
# DO NOT CHANGE
import cv2
import numpy as np
import glob
import time

## 1. Fill x_test and y_test:
x_test = []
y_test = []
fonts = [ 'IBM Plex Sans Arabic', 'Lemonada', 'Marhey', 'Scheherazade New']

for font in fonts:
    for filename in sorted(glob.glob(f'.\content\\test\\{font}\\*.jpeg')):
        img = cv2.imread(filename, cv2.COLOR_BGR2RGB)
        x_test.append(img)
        y_test.append(fonts.index(font))

# 2. Convert them to Numpy arrays:
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)


print("shape of x_data:", x_test.shape, "shape of y_data:", y_test.shape)
assert set(y_test) == {0, 1, 2, 3}

shape of x_data: (8, 1181, 1181, 3) shape of y_data: (8,)


Here you define your predict function following this specific interface:

In [45]:
'''
This function takes an RGB image of dimensions (1181, 1181, 3) from the test set and returns integer prediction ∈ {0,1,2,3}
'''

kmeans_model_filename = "kmeans_model.pkl"
svm_model_filename = "svm_model.pkl"
def make_prediction(image):
    # DEFINE YOUR FUNCTION HERE AND DO NOT CHANGE ANYTHING ELSE IN THE NOTEBOOK
    # Load the KMeans model
    with open(kmeans_model_filename, 'rb') as file:
        loaded_kmeans_model = pickle.load(file)
    
    # Load the SVM model
    with open(svm_model_filename, 'rb') as file:
        loaded_svm_model = pickle.load(file)
    histogram = []
    preprocessed_image = remove_noise22(image, dilation_kernel=None)
    descriptors = extract_sift_features(preprocessed_image)
    if descriptors is not None and descriptors.shape[0] > 0 and descriptors.shape[1] > 0:
         nearest_clusters = pairwise_distances_argmin_min(descriptors, loaded_kmeans_model.cluster_centers_)[0]
         histogram, _ = np.histogram(nearest_clusters, bins=np.arange(loaded_kmeans_model.n_clusters + 1))
                
    X_test = histogram
    
    y_pred = loaded_svm_model.predict(X_test)
    return y_pred



# Fill your team number here
TEAM_NUM = "17"

Now let's compute the accuracy of the model:

In [48]:
# DO NOT CHANGE
y_pred = []

start_time = time.time()
for x in x_test:
    assert x.shape == (1181, 1181, 3)
    ŷ = make_prediction(x)
    y_pred.append(ŷ)
end_time = time.time()

y_pred = np.asarray(y_pred)
accuracy = np.mean(y_pred == y_test)
total_time = end_time - start_time
print(f"Team {TEAM_NUM} got accuracy: {accuracy:.2%}")
print(f"Team {TEAM_NUM} got runtime: {total_time:.2%}")

ValueError: Expected 2D array, got 1D array instead:
array=[ 7.  2. 11.  1.  2. 12.  2.  2.  9.  0. 10. 15.  8.  8.  8.  2.  2.  4.
 10. 13.  3. 11.  3.  6.  8.  1.  2. 26. 14.  4.  9.  8. 15.  1.  0.  2.
  0.  6.  5.  8.  1. 10.  5. 11. 27. 16.  5.  7.  2.  5. 10.  2.  3.  9.
  0.  5.  0. 18.  0.  3.  5.  7. 13.  8.  0.  7.  8.  1. 10. 10.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.